# MMOB Project

# import

in this cell, relevant libraries are imported, as well as the datafile imported from our github and a dict that will store all results during the project 

In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
data_file ='https://raw.githubusercontent.com/DunodMax/MMOB/main/Dataset.txt'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC
database = db.Database('LPMC', LPMC)
all_results = {}

/Users/maximedunod/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Model 0

all useful columns for this model are used as variables

In [2]:
travel_mode=Variable('travel_mode')
dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')
pt_interchanges=Variable('pt_interchanges')
dur_driving=Variable('dur_driving')
cost_transit=Variable('cost_transit')
cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
dur_walking=Variable('dur_walking')
dur_cycling=Variable('dur_cycling')

Trip duration for each mode are computed ( see report for details about equations)
Cost for each mode with a cost are computed ( cycling and walking is considered free)

In [3]:
time_walk=dur_walking
time_cycle=dur_cycling
time_pt=dur_pt_access+dur_pt_rail+dur_pt_bus+dur_pt_int
time_drive=dur_driving

cost_drive=cost_driving_ccharge+cost_driving_fuel
cost_pt=cost_transit

There is 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [4]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

utility functions

In [5]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

In [6]:
av = {1: 1, 2: 1, 3: 1, 4:1}

estimation of the parameters for Model 0, all relevant parameters are printed

In [7]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

[17:15:42] < Warning >   Cannot read file __Model_0.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.832163,0.107574,-35.623351,0.0
asc_drive,-1.235707,0.080400,-15.369411,0.0
asc_pt,-0.528767,0.054766,-9.655006,0.0
beta_cost,-0.169780,0.013108,-12.952753,0.0
beta_time,-5.450540,0.202684,-26.891767,0.0


In [8]:
res.compileEstimationResults(all_results)

,Model0
Number of estimated parameters,5.000000
Sample size,5000.000000
Final log likelihood,-4566.446492
Akaike Information Criterion,9142.892983
Bayesian Information Criterion,9175.478949
asc_cycle,-3.832163
asc_drive,-1.235707
asc_pt,-0.528767
beta_cost,-0.169780
beta_time,-5.450540


In [9]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

# Model 1 

### Time specification

We will try a specification where the the cost coeffecient is Generic and the time coefficient is alternative specific

Again 4 different travel mode, 4 utility functions and 3 alternative specific constant.
This time we have the generic parameter for cost and a 4 new specific parameter for travel time

In [10]:
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)

New utility functions :

In [11]:
v_walk_model1 = beta_time_walk * time_walk  
v_cycle_model1 = asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1 = asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1 = asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

Estimation of the parameters for Model 1, all relevant parameters are printed

In [12]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model1_time_specification'
all_results['Model1_time_specification'] = biogeme_model1.estimate()
results_alt_spec_time = biogeme_model1.estimate()
results_alt_spec_time.getEstimatedParameters()

[17:15:46] < Warning >   Cannot read file __Model1_time_specification.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.509593,0.191028,-23.606921,0.0
asc_drive,-1.885701,0.122740,-15.363361,0.0
asc_pt,-2.339709,0.125798,-18.598939,0.0
beta_cost,-0.146061,0.014850,-9.835754,0.0
beta_time_cycle,-6.121448,0.542680,-11.280025,0.0
beta_time_drive,-6.445066,0.391345,-16.469010,0.0
beta_time_pt,-3.514554,0.258207,-13.611367,0.0
beta_time_walk,-8.204816,0.364365,-22.518121,0.0


In [13]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification
Number of estimated parameters,5,8
Sample size,5000,5000
Final log likelihood,-4566.446492,-4264.834249
Akaike Information Criterion,9142.892983,8545.668498
Bayesian Information Criterion,9175.478949,8597.806043
asc_cycle,-3.832163,-4.509598
asc_drive,-1.235707,-1.885704
asc_pt,-0.528767,-2.339712
beta_cost,-0.16978,-0.146061
beta_time,-5.45054,


### Cost specification

We compare with the model where we specify the price, to be sure to have the most interesting model for the future models. (walk and cycling don't have cost, so only 2 specific parameter for cost)

In [14]:
beta_cost_drive = Beta('beta_cost_drive', 0, None, None, 0)
beta_cost_pt = Beta('beta_cost_pt', 0, None, None, 0)

New utility function

In [15]:
v_walk_model1_spec_cost = beta_time * time_walk  
v_cycle_model1_spec_cost = asc_cycle + beta_time * time_cycle 
v_pt_model1_spec_cost = asc_pt + beta_time * time_pt + beta_cost_pt * cost_pt
v_drive_model1_spec_cost = asc_drive + beta_time * time_drive + beta_cost_drive * cost_drive

In [16]:
V_model1_spec_cost = {1: v_walk_model1_spec_cost , 2: v_cycle_model1_spec_cost, 3: v_pt_model1_spec_cost, 4: v_drive_model1_spec_cost}
logprob_model1_spec_cost = models.loglogit(V_model1_spec_cost, av, travel_mode)
biogeme_model1_spec_cost = bio.BIOGEME(database, logprob_model1_spec_cost)
biogeme_model1_spec_cost.modelName = 'Model_1_cost_specifiaction'
all_results['Model1_cost_specification'] = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost.getEstimatedParameters()

[17:15:48] < Warning >   Cannot read file __Model_1_cost_specifiaction.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.766977,0.107734,-34.965548,0.000000
asc_drive,-1.137426,0.080194,-14.183498,0.000000
asc_pt,-0.831978,0.064856,-12.828129,0.000000
beta_cost_drive,-0.214644,0.021272,-10.090610,0.000000
beta_cost_pt,0.061554,0.028466,2.162411,0.030587
beta_time,-5.411681,0.202913,-26.670010,0.000000


In [17]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification
Number of estimated parameters,5,8,6
Sample size,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834249,-4509.365231
Akaike Information Criterion,9142.892983,8545.668498,9030.730462
Bayesian Information Criterion,9175.478949,8597.806043,9069.833621
asc_cycle,-3.832163,-4.509598,-3.766977
asc_drive,-1.235707,-1.885704,-1.137426
asc_pt,-0.528767,-2.339712,-0.831978
beta_cost,-0.16978,-0.146061,
beta_time,-5.45054,,-5.411681


### Comparaison

we compare the Akaike Information Criterion (AIC)

In [18]:
print("Model 0 PIC value :",results_generic.data.akaike)
print("Model 1, time alternative specification PIC value :",results_alt_spec_time.data.akaike)
print("Model 1, cost alternative specification PIC value :",results_alt_spec_cost.data.akaike)

Model 0 PIC value : 9142.89298347719
Model 1, time alternative specification PIC value : 8545.668496844653
Model 1, cost alternative specification PIC value : 9030.7304615338


we compare the Bayesian Information Criterion (BIC)

In [19]:
print("Model 0 PIC value :",results_generic.data.bayesian)
print("Model 1, alternative specification travel time, PIC value :",results_alt_spec_time.data.bayesian)
print("Model 1, alternative specification cost, PIC value :",results_alt_spec_cost.data.bayesian)

Model 0 PIC value : 9175.478949434271
Model 1, alternative specification travel time, PIC value : 8597.806042375983
Model 1, alternative specification cost, PIC value : 9069.833620682297


According to the AIC and the BIC, the best model is the one wih the alternative specific travel time coefficient, and the generic travel cost coefficient.

# Model 2

In [20]:
LPMC.columns

Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
       'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time', 'age', 'female', 'driving_license', 'car_ownership',
       'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
       'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'pt_interchanges',
       'dur_driving', 'cost_transit', 'cost_driving_fuel',
       'cost_driving_ccharge', 'driving_traffic_percent'],
      dtype='object')

In [21]:
model_base=biogeme_model1

In [22]:
#we're adding the attribute distance to our model 
#we're adding the interaction between age and distance
pt_interchanges=Variable('pt_interchanges')
driving_license=Variable('driving_license')

In [23]:
#defining associated coefficients
beta_pt_inter= Beta('beta_pt_inter', 0, None, None, 0)
beta_license= Beta('beta_license', 0, None, None, 0)

In [24]:
#the specification, we only consider the alternative distance for walking
#the interaction distance age is considered just for public trasnport
v_walk_model2 = v_walk_model1
v_cycle_model2 = v_cycle_model1
v_pt_model2 = v_pt_model1 + beta_pt_inter*pt_interchanges
v_drive_model2 = v_drive_model1 + asc_drive*beta_license*driving_license

In [25]:
#the estimation results
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model_2'] = biogeme_model2.estimate()
results_model2 = biogeme_model2.estimate()
results_model2.getEstimatedParameters()

[17:15:49] < Warning >   Cannot read file __Model_2.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.566489,0.196947,-23.186435,0.000000
asc_drive,-2.782432,0.138492,-20.090922,0.000000
asc_pt,-2.421769,0.131403,-18.430079,0.000000
beta_cost,-0.140456,0.013889,-10.112732,0.000000
beta_license,-0.517648,0.027952,-18.519340,0.000000
beta_pt_inter,-0.107552,0.089736,-1.198541,0.230707
beta_time_cycle,-6.305828,0.559567,-11.269120,0.000000
beta_time_drive,-7.003842,0.411151,-17.034715,0.000000
beta_time_pt,-3.541166,0.282515,-12.534425,0.000000
beta_time_walk,-8.376052,0.377032,-22.215752,0.000000


In [26]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2
Number of estimated parameters,5,8,6,10
Sample size,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834249,-4509.365231,-4039.286551
Akaike Information Criterion,9142.892983,8545.668498,9030.730462,8098.573101
Bayesian Information Criterion,9175.478949,8597.806043,9069.833621,8163.745033
asc_cycle,-3.832163,-4.509598,-3.766977,-4.566495
asc_drive,-1.235707,-1.885704,-1.137426,-2.782437
asc_pt,-0.528767,-2.339712,-0.831978,-2.421776
beta_cost,-0.16978,-0.146061,,-0.140457
beta_time,-5.45054,,-5.411681,


In [27]:
#we test base_model against model_2 where base_model is a restricted version of model_2 suing the looglikehood
#ratio test

In [28]:
#we test them using the null hypothesis that our base model is the true model

In [29]:
results_base=model_base.estimate()

In [30]:
results_model2.likelihood_ratio_test(results_base, 0.05) #level of the statistics for a level of significance of 5%?

LRTuple(message='H0 can be rejected at level 5.0%', statistic=451.09539568599666, threshold=5.991464547107979)

In [31]:
#then depending on if the value I obtained was higher or lower than a certain number, we reject or accept the null hypothesis

In [32]:
from scipy.stats import chi2
threshold = chi2.ppf(.95, 2)
threshold #just in case we might want to compare to this threshold

5.991464547107979

# Model 3

We will conduct a non-linear transformation of time

In [33]:
model_base=biogeme_model2
log_time_walk = database.DefineVariable('log_time_walk', log(time_walk))
log_time_cycle = database.DefineVariable('log_time_cycle', log(time_cycle))
log_time_pt = database.DefineVariable('log_time_pt', log(time_pt))
log_time_drive = database.DefineVariable('log_time_drive', log(time_drive))
log_beta_time_drive = Beta('log_beta_time_drive', 0, None, None, 0)
log_beta_time_pt = Beta('log_beta_time_pt', 0, None, None, 0)
log_beta_time_walk = Beta('log_beta_time_walk', 0, None, None, 0)
log_beta_time_cycle = Beta('log_beta_time_cycle', 0, None, None, 0)

In [34]:
v_walk_model3 = log_beta_time_walk * log_time_walk
v_cycle_model3 = asc_cycle + log_beta_time_cycle * log_time_cycle
v_pt_model3 = asc_pt + log_beta_time_pt * log_time_pt + beta_cost * cost_pt + beta_pt_inter*pt_interchanges
v_drive_model3 = asc_drive + log_beta_time_drive * log_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license

In [35]:
V_model3 = {1: v_walk_model3 , 2: v_cycle_model3, 3: v_pt_model3, 4: v_drive_model3}
logprob_model3 = models.loglogit(V_model3, av, travel_mode)
biogeme_model3 = bio.BIOGEME(database, logprob_model3)
biogeme_model3.modelName = 'Model_3'
all_results['Model_3'] = biogeme_model3.estimate()
results_model3 = biogeme_model3.estimate()
results_model3.getEstimatedParameters()

[17:15:51] < Warning >   Cannot read file __Model_3.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.440357,0.227036,-10.748772,0.000000
asc_drive,-0.680104,0.180008,-3.778185,0.000158
asc_pt,1.677853,0.147456,11.378653,0.000000
beta_cost,-0.137105,0.013208,-10.380838,0.000000
beta_license,-2.101083,0.537274,-3.910636,0.000092
beta_pt_inter,-0.093153,0.073977,-1.259214,0.207953
log_beta_time_cycle,-2.464649,0.131859,-18.691558,0.000000
log_beta_time_drive,-2.344192,0.101472,-23.101947,0.000000
log_beta_time_pt,-2.025236,0.120497,-16.807409,0.000000
log_beta_time_walk,-4.598998,0.133120,-34.547665,0.000000


In [36]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3
Number of estimated parameters,5,8,6,10,10
Sample size,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834249,-4509.365231,-4039.286551,-3975.362089
Akaike Information Criterion,9142.892983,8545.668498,9030.730462,8098.573101,7970.724177
Bayesian Information Criterion,9175.478949,8597.806043,9069.833621,8163.745033,8035.896109
asc_cycle,-3.832163,-4.509598,-3.766977,-4.566495,-2.440357
asc_drive,-1.235707,-1.885704,-1.137426,-2.782437,-0.680104
asc_pt,-0.528767,-2.339712,-0.831978,-2.421776,1.677853
beta_cost,-0.16978,-0.146061,,-0.140457,-0.137105
beta_time,-5.45054,,-5.411681,,


We will compare Model_2 and Model_3 with a cox test, thus we will create a generic model

In [37]:
v_walk_model_generic = log_beta_time_walk * log_time_walk + beta_time_walk * time_walk
v_cycle_model_generic = asc_cycle + log_beta_time_cycle * log_time_cycle + beta_time_cycle * time_cycle
v_pt_model_generic = asc_pt + log_beta_time_pt * log_time_pt + beta_cost * cost_pt + beta_pt_inter*pt_interchanges + beta_time_pt * time_pt
v_drive_model_generic = asc_drive + log_beta_time_drive * log_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license + beta_time_drive * time_drive

In [38]:
V_model_generic = {1: v_walk_model_generic , 2: v_cycle_model_generic, 3: v_pt_model_generic, 4: v_drive_model_generic}
logprob_model_generic = models.loglogit(V_model_generic, av, travel_mode)
biogeme_model_generic = bio.BIOGEME(database, logprob_model_generic)
biogeme_model_generic.modelName = 'Model_generic'
all_results['Model_generic'] = biogeme_model_generic.estimate()
results_model_generic = biogeme_model_generic.estimate()
results_model_generic.getEstimatedParameters()

[17:15:53] < Warning >   Cannot read file __Model_generic.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.092631,0.676585,-3.092932,1.981897e-03
asc_drive,-0.311548,0.360765,-0.863575,3.878215e-01
asc_pt,2.500510,0.466756,5.357207,8.451816e-08
beta_cost,-0.129692,0.013880,-9.343881,0.000000e+00
beta_license,-4.630841,5.359639,-0.864021,3.875764e-01
beta_pt_inter,-0.035905,0.085233,-0.421256,6.735685e-01
beta_time_cycle,-2.579461,0.793375,-3.251252,1.148979e-03
beta_time_drive,-3.075970,0.576765,-5.333139,9.652933e-08
beta_time_pt,-2.944321,0.534183,-5.511823,3.551369e-08
beta_time_walk,-1.610458,0.464996,-3.463383,5.334294e-04


In [39]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3,Model_generic
Number of estimated parameters,5,8,6,10,10,14
Sample size,5000,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834249,-4509.365231,-4039.286551,-3975.362089,-3948.931227
Akaike Information Criterion,9142.892983,8545.668498,9030.730462,8098.573101,7970.724177,7925.862453
Bayesian Information Criterion,9175.478949,8597.806043,9069.833621,8163.745033,8035.896109,8017.103158
asc_cycle,-3.832163,-4.509598,-3.766977,-4.566495,-2.440357,-2.092628
asc_drive,-1.235707,-1.885704,-1.137426,-2.782437,-0.680104,-0.311544
asc_pt,-0.528767,-2.339712,-0.831978,-2.421776,1.677853,2.500508
beta_cost,-0.16978,-0.146061,,-0.140457,-0.137105,-0.129692
beta_time,-5.45054,,-5.411681,,,


In [40]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=180.71064703482534, threshold=9.487729036781154)

In [41]:
model_base=biogeme_model3
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=52.86172453334257, threshold=9.487729036781154)

In this case the cox-test tells us to find better models, we will thus use a box-cox transform

In [43]:
lambda_boxcox = Beta('lambda_boxcox', 1, None, None, 0)
boxcox_time_walk = models.boxcox(time_walk, lambda_boxcox)
boxcox_time_cycle = models.boxcox(time_cycle, lambda_boxcox)
boxcox_time_pt = models.boxcox(time_pt, lambda_boxcox)
boxcox_time_drive = models.boxcox(time_drive , lambda_boxcox)
beta_elapsed_time_walk = Beta('beta_elapsed_walk', 0, None, None, 0)
beta_elapsed_time_cycle = Beta('beta_elapsed_cycle', 0, None, None, 0)
beta_elapsed_time_pt = Beta('beta_elapsed_time_pt', 0, None, None, 0)
beta_elapsed_time_drive = Beta('beta_elapsed_time_drive', 0, None, None, 0)
v_walk_model_boxcox = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model_boxcox  = asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model_boxcox  = asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt + beta_pt_inter*pt_interchanges
v_drive_model_boxcox  = asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license

In [44]:
V_model_boxcox = {1: v_walk_model_boxcox , 2: v_cycle_model_boxcox, 3: v_pt_model_boxcox, 4: v_drive_model_boxcox}
logprob_model_boxcox = models.loglogit(V_model_boxcox, av, travel_mode)
biogeme_model_boxcox = bio.BIOGEME(database, logprob_model_boxcox)
biogeme_model_boxcox.modelName = 'Model_boxcox'
all_results['Model_boxcox'] = biogeme_model_boxcox.estimate()
results_model_boxcox = biogeme_model_boxcox.estimate()
results_model_boxcox.getEstimatedParameters()

[17:16:44] < Warning >   Cannot read file __Model_boxcox.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.794017,0.287787,-9.708621,0.000000e+00
asc_drive,-1.325820,0.237166,-5.590256,2.267351e-08
asc_pt,1.641993,0.177913,9.229193,0.000000e+00
beta_cost,-0.129339,0.013804,-9.370027,0.000000e+00
beta_elapsed_cycle,-3.661618,0.287125,-12.752709,0.000000e+00
beta_elapsed_time_drive,-3.701045,0.275373,-13.440099,0.000000e+00
beta_elapsed_time_pt,-2.718524,0.190871,-14.242719,0.000000e+00
beta_elapsed_walk,-5.667674,0.237039,-23.910320,0.000000e+00
beta_license,-1.091700,0.190782,-5.722244,1.051262e-08
beta_pt_inter,-0.147415,0.080668,-1.827429,6.763534e-02


In [45]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3,Model_generic,Model_boxcox
Number of estimated parameters,5,8,6,10,10,14,11
Sample size,5000,5000,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834249,-4509.365231,-4039.286551,-3975.362089,-3948.931227,-3953.048852
Akaike Information Criterion,9142.892983,8545.668498,9030.730462,8098.573101,7970.724177,7925.862453,7928.097705
Bayesian Information Criterion,9175.478949,8597.806043,9069.833621,8163.745033,8035.896109,8017.103158,7999.78683
asc_cycle,-3.832163,-4.509598,-3.766977,-4.566495,-2.440357,-2.092628,-2.794017
asc_drive,-1.235707,-1.885704,-1.137426,-2.782437,-0.680104,-0.311544,-1.32582
asc_pt,-0.528767,-2.339712,-0.831978,-2.421776,1.677853,2.500508,1.641993
beta_cost,-0.16978,-0.146061,,-0.140457,-0.137105,-0.129692,-0.129339
beta_time,-5.45054,,-5.411681,,,,


In [46]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_boxcox.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=172.4753942531106, threshold=3.841458820694124)